# FTX reader

In [88]:
import matplotlib
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px

import sys
sys.path.append('../')
from history_script import *

import os
directory='C:/Users/david/puffin/DerivativeArbitrage/Runtime/live_parquets/'
end_time=datetime.today().replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(weeks=1)
modes=[('h','high'),('c','close'),('l','low')]
coins=['DOGE']

In [89]:
futures_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='1h',coin_list=coins)

subaccount list: margintest / 
funding_history: DOGE-PERP
rate_history: DOGE-PERP
rate_history: DOGE-1231
price_history: DOGE/USD
borrow_history: DOGE
borrow_history: USD


### rates markets

In [90]:
prices=futures_df[[c for c in futures_df.columns if any(string in c for string in ['price/c'])]]

short_rates=futures_df[[c for c in futures_df.columns if any(string in c for string in ['PERP/rate/c','rate/borrow'])]]
short_rates=short_rates.ewm(halflife=timedelta(hours=12),times=short_rates.index).mean()

long_rates=futures_df[[c for c in futures_df.columns if all(['/rate/c' in c,not ('PERP' in c)])]]
long_rates=long_rates.ewm(halflife=timedelta(hours=1),times=short_rates.index).mean()

for c in coins:
    short_rates[c+'/long_carry']=short_rates[c+'-PERP/rate/c']-short_rates['USD/rate/borrow']
    short_rates[c+'/short_carry']=-short_rates[c+'-PERP/rate/c']-short_rates[c+'/rate/borrow']

pd.concat([prices,short_rates,long_rates],axis=0).iplot(secondary_y=list(short_rates.columns)+list(long_rates.columns))

### carry

In [91]:
pd.concat([prices,short_rates.filter(like='carry')],axis=0).iplot(secondary_y=list(prices.columns))

# stopout: show min/max of premiums

In [92]:
end_time=datetime.today().replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(days=1)
granular_df=ftx_history(dirname='',start=start_time,end=end_time,timeframe='15s',coin_list=coins)

subaccount list: margintest / 
funding_history: DOGE-PERP
rate_history: DOGE-PERP
rate_history: DOGE-1231
price_history: DOGE/USD
borrow_history: DOGE
borrow_history: USD


In [80]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [81]:
granular_df['premium']=compute_premium(granular_df)
prices=granular_df[[c+'/price/c' for c in coins]]

In [82]:
premium=compute_premium(granular_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
pd.concat([prices,ewMax],axis=0).iplot(secondary_y=list(prices.columns))

In [84]:
premium=compute_premium(granular_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
pd.concat([prices,ewMin],axis=0).iplot(secondary_y=list(prices.columns))